In [1]:
from datetime import datetime
from IPython.display import display, Markdown

todays_date = str(datetime.now().date())

display(
    Markdown(
        "# Notebook - Generate ESM variable length ESM-3 embeddings of OVA and RBD antibody sequences"
    )
)
display(Markdown("Author: Lena Erlach"))
display(Markdown("Created: 2024-08-21"))
display(Markdown(f"Last modified: {todays_date}"))

# Notebook - Generate ESM variable length ESM-3 embeddings of OVA and RBD antibody sequences

Author: Lena Erlach

Created: 2024-08-21

Last modified: 2024-08-28

In [2]:
import numpy as np
import pandas as pd
import torch
import huggingface_hub
import configparser, os, sys

from tqdm import tqdm
import pickle

from esm.models.esm3 import ESM3

from esm.sdk.api import (
    ESMProtein,
    SamplingConfig,
)

UTILS_DIR = os.path.join("../../src")
sys.path.append(UTILS_DIR)



######### CUSTOM FUNCTIONS ###########

# ESM3 embedding function
def esm3_embed_seqs(seq_df, chain_type, save_plm, out_path_PLM, model):

    failed = []

    # loop through seqs
    for index, row in tqdm(seq_df.iterrows(), total=seq_df.shape[0]):

        # def seq
        if chain_type == "H":
            seq = row["VDJ_aaSeq"]
        elif chain_type == "L":
            seq = row["VJ_aaSeq"]
        
        try:
            # Generate embeddings
            protein = ESMProtein(sequence=seq)
            protein_tensor = model.encode(protein)

            output = model.forward_and_sample(protein_tensor, SamplingConfig(return_per_residue_embeddings=True))
            embeddings = output.per_residue_embedding.to("cpu").detach().numpy()
            # save embedding as pickle
            file_name = "{}_{}.p".format(row["seq_id"], chain_type)

            if save_plm is True:
                with open(os.path.join(out_path_PLM, file_name), "wb") as fh:
                    # print(f'Saving {row["seq_id"]} to {os.path.join(out_path_PLM, file_name)}')
                    pickle.dump(embeddings, fh)

        except Exception as e:
            print(f'Failed: {row["seq_id"]}')
            print(e)
            failed.append(row["seq_id"])

    return failed


/data/cb/scratch/lenae/software/Anaconda/envs/esm3-env/lib/python3.10/site-packages/esm/utils/structure/protein_structure.py:69: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/data/cb/scratch/lenae/software/Anaconda/envs/esm3-env/lib/python3.10/site-packages/esm/utils/structure/protein_structure.py:164: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/data/cb/scratch/lenae/software/Anaconda/envs/esm3-env/lib/python3.10/site-packages/esm/utils/structure/protein_structure.py:213: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


In [5]:
##### Setup the GPU support:
cuda_dev_num = 7
if torch.cuda.is_available():
    dev = "cuda:{}".format(cuda_dev_num)
else:
    dev = "cpu"
device = torch.device(dev)
torch.cuda.set_device(device)

# huggingface_hub.login()  # will prompt you to get an API key and accept the ESM3 license.
model = ESM3.from_pretrained("esm3_sm_open_v1", device=device)


/data/cb/scratch/lenae/software/Anaconda/envs/esm3-env/lib/python3.10/site-packages/esm/pretrained.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.lo

### OVA sequences VH_VL embeddings 

In [3]:
# setup parser for the config file ../../example_config_file.txt
CONFIG_PATH = "../../config_file.txt"
config = configparser.ConfigParser()
config.read(CONFIG_PATH)
ROOT_DIR = config["ROOT"]["ROOT_DIR"]

dataset = config["SETUP"]["DATASET"]


# Set input path Sequences
seq_df_inputPath = os.path.join(ROOT_DIR, config["PATHS"]["SEQ_DF"])
seq_df = pd.read_csv(seq_df_inputPath)


seq_col = "VDJ_VJ_aaSeq"  # column name of the sequence to filter for (VDJ_VJ_aaSeq, VDJ_aaSeq, ...)
if seq_col == "VDJ_VJ_aaSeq": 
    seq_df = seq_df[seq_df.seq_complete == True]

# filter 192 for OVA
if dataset == "OVA":
    seq_df.drop(192, inplace=True)

# embedding paths for VH_VL embeddings
out_path_PLM = os.path.join(ROOT_DIR, f'data/processed/embeddings/{dataset}/ESM3')
if not os.path.isdir(out_path_PLM):
    os.makedirs(out_path_PLM)

In [5]:
### Generate sequence embeddings
f_ids = []
for c_type in ["H", "L"]:
    print(f'Embed {c_type} chains')
    failed = esm3_embed_seqs(seq_df, chain_type=c_type, save_plm = True, out_path_PLM=out_path_PLM, model=model)
    f_ids.append(failed)

### RBD sequences VH_VL embedding

In [4]:
# setup parser for the config file 
CONFIG_PATH = "../../config_file_RBD.txt"
config = configparser.ConfigParser()
config.read(CONFIG_PATH)
ROOT_DIR = config["ROOT"]["ROOT_DIR"]

dataset = config["SETUP"]["DATASET"]


# Set input path Sequences
seq_df_inputPath = os.path.join(ROOT_DIR, config["PATHS"]["SEQ_DF"])
seq_df = pd.read_csv(seq_df_inputPath)


seq_col = "VDJ_VJ_aaSeq"  # column name of the sequence to filter for (VDJ_VJ_aaSeq, VDJ_aaSeq, ...)
# if seq_col == "VDJ_VJ_aaSeq": 
#     seq_df = seq_df[seq_df.seq_complete == True]



# embedding paths for VH_VL embeddings
out_path_PLM = os.path.join(ROOT_DIR, f'data/processed/embeddings/{dataset}/ESM3')
if not os.path.isdir(out_path_PLM):
    os.makedirs(out_path_PLM)


In [5]:
### Generate sequence embeddings
f_ids = []
for c_type in ["H", "L"]:
    print(f'Embed {c_type} chains')
    failed = esm3_embed_seqs(seq_df, chain_type=c_type, save_plm = True, out_path_PLM=out_path_PLM, model=model)
    f_ids.append(failed)

Embed H chains


100%|██████████| 3784/3784 [03:35<00:00, 17.56it/s]


Embed L chains


  1%|          | 33/3784 [00:01<03:05, 20.27it/s]

Failed: 30_s1
'float' object has no attribute 'replace'


...

100%|█████████▉| 3768/3784 [03:24<00:00, 20.85it/s]

Failed: 3765_s2
'float' object has no attribute 'replace'


100%|█████████▉| 3774/3784 [03:24<00:00, 21.43it/s]

Failed: 3771_s2
'float' object has no attribute 'replace'
Failed: 3774_s2
'float' object has no attribute 'replace'


100%|██████████| 3784/3784 [03:25<00:00, 18.44it/s]
